In [1]:
#1)import all the stuff
import numpy as np
import pandas as pd
from  sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.model_selection import StratifiedKFold

In [18]:
#set view stuff (only needed for draft in Jupyter notebook)
pd.set_option('display.max_columns', 100)

In [19]:
#2)import data scraped from web and saved to csv
movie_data = pd.read_csv("IMDBTop250.csv")
movie_data.head()

,Unnamed: 0,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,Released,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes
0,0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Nominated for 7 Oscars. Another 19 wins & 30 n...,USA,Frank Darabont,"Crime, Drama",English,80.0,Two imprisoned men bond over a number of years...,https://images-na.ssl-images-amazon.com/images...,R,14 Oct 1994,True,142 min,The Shawshank Redemption,movie,"Stephen King (short story ""Rita Hayworth and S...",1994,tt0111161,9.3,"1,786,262"
1,1,"Marlon Brando, Al Pacino, James Caan, Richard ...",Won 3 Oscars. Another 23 wins & 27 nominations.,USA,Francis Ford Coppola,"Crime, Drama","English, Italian, Latin",100.0,The aging patriarch of an organized crime dyna...,https://images-na.ssl-images-amazon.com/images...,R,24 Mar 1972,True,175 min,The Godfather,movie,"Mario Puzo (screenplay), Francis Ford Coppola ...",1972,tt0068646,9.2,"1,219,320"
2,2,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",Won 6 Oscars. Another 10 wins & 20 nominations.,USA,Francis Ford Coppola,"Crime, Drama","English, Italian, Spanish, Latin, Sicilian",80.0,The early life and career of Vito Corleone in ...,https://images-na.ssl-images-amazon.com/images...,R,20 Dec 1974,True,202 min,The Godfather: Part II,movie,"Francis Ford Coppola (screenplay), Mario Puzo ...",1974,tt0071562,9.0,"839,135"
3,3,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",Won 2 Oscars. Another 146 wins & 142 nominations.,"USA, UK",Christopher Nolan,"Action, Crime, Drama","English, Mandarin",82.0,When the menace known as the Joker wreaks havo...,https://images-na.ssl-images-amazon.com/images...,PG-13,18 Jul 2008,True,152 min,The Dark Knight,movie,"Jonathan Nolan (screenplay), Christopher Nolan...",2008,tt0468569,9.0,"1,754,213"
4,4,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",Nominated for 3 Oscars. Another 16 wins & 8 no...,USA,Sidney Lumet,"Crime, Drama",English,NaN,A jury holdout attempts to prevent a miscarria...,https://images-na.ssl-images-amazon.com/images...,APPROVED,01 Apr 1957,True,96 min,12 Angry Men,movie,"Reginald Rose (story), Reginald Rose (screenplay)",1957,tt0050083,8.9,"481,606"


In [20]:
#3)operationalize y into two categories: "amazing"=1 and "good"=0
#this is the target variable
is_amazing = movie_data["imdbRating"].apply(lambda x: 0 if x <= np.median(movie_data["imdbRating"]) else 1)

In [21]:
#4)clean data and make X for modeling
#strip "min" from runtime (maybe retitle as runtime_minutes?)
movie_data["Runtime"] = movie_data["Runtime"].str.replace(" min","")

In [22]:
#turn runtime into number
movie_data["Runtime"] = movie_data["Runtime"].apply(int)

In [23]:
#make director dummies:
dir_dummies = pd.get_dummies(movie_data["Director"], prefix="Dir")

In [24]:
dir_dummies.head()

,"Dir_Aamir Khan, Amole Gupte",Dir_Adam Elliot,Dir_Akira Kurosawa,Dir_Alejandro G. Iñárritu,Dir_Alfred Hitchcock,Dir_Andrei Tarkovsky,Dir_Andrew Stanton,"Dir_Andrew Stanton, Lee Unkrich",Dir_Anurag Kashyap,Dir_Asghar Farhadi,Dir_Ashutosh Gowariker,Dir_Billy Wilder,Dir_Brian De Palma,Dir_Bryan Singer,"Dir_Byron Howard, Rich Moore, Jared Bush",Dir_Carl Theodor Dreyer,Dir_Carol Reed,Dir_Chan-wook Park,Dir_Charles Chaplin,Dir_Christopher Nolan,Dir_Clint Eastwood,"Dir_Clyde Bruckman, Buster Keaton",Dir_Curtis Hanson,Dir_Damien Chazelle,Dir_Damián Szifrón,Dir_Danny Boyle,Dir_Darren Aronofsky,Dir_David Fincher,Dir_David Lean,Dir_David Lynch,Dir_David Yates,"Dir_Dean DeBlois, Chris Sanders",Dir_Denis Villeneuve,Dir_Elem Klimov,Dir_Elia Kazan,"Dir_Ethan Coen, Joel Coen",Dir_F.W. Murnau,Dir_Federico Fellini,"Dir_Fernando Meirelles, Kátia Lund",Dir_Florian Henckel von Donnersmarck,Dir_Francis Ford Coppola,Dir_Frank Capra,Dir_Frank Darabont,Dir_François Truffaut,Dir_Fritz Lang,Dir_Gavin O'Connor,Dir_George Lucas,Dir_George Miller,Dir_George Roy Hill,Dir_Gillo Pontecorvo,...,Dir_Peter Weir,Dir_Priyadarshan,Dir_Quentin Tarantino,"Dir_Quentin Tarantino, Eli Roth",Dir_Rajkumar Hirani,Dir_Rajkumar Santoshi,Dir_Rakeysh Omprakash Mehra,Dir_Ramesh Sippy,Dir_Richard Attenborough,Dir_Richard Kelly,Dir_Richard Linklater,Dir_Richard Marquand,Dir_Ridley Scott,Dir_Rob Reiner,Dir_Robert Aldrich,Dir_Robert Mulligan,Dir_Robert Zemeckis,Dir_Roberto Benigni,"Dir_Roger Allers, Rob Minkoff",Dir_Roman Polanski,Dir_Ron Howard,Dir_Sam Mendes,Dir_Sean Penn,Dir_Sergio Leone,Dir_Sidney Lumet,"Dir_Stanley Donen, Gene Kelly",Dir_Stanley Kramer,Dir_Stanley Kubrick,Dir_Steve McQueen,Dir_Steven Spielberg,Dir_Stuart Rosenberg,Dir_Tate Taylor,Dir_Terry George,Dir_Terry Gilliam,"Dir_Terry Gilliam, Terry Jones",Dir_Terry Jones,Dir_Thomas Vinterberg,Dir_Tom McCarthy,Dir_Tony Kaye,"Dir_Victor Fleming, George Cukor, Mervyn LeRoy, Norman Taurog, King Vidor","Dir_Victor Fleming, George Cukor, Sam Wood",Dir_Vittorio De Sica,"Dir_Wai-Keung Lau, Alan Mak",Dir_Wes Anderson,Dir_William Wyler,Dir_Wolfgang Petersen,Dir_Woody Allen,Dir_Yasujirô Ozu,Dir_Yavuz Turgul,Dir_Çagan Irmak
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
#fill nan in rated col
movie_data["Rated"].fillna("NOT RATED", inplace=True)

In [28]:
#make rated dummies:
rated_dummies = pd.get_dummies(movie_data["Rated"])

In [29]:
rated_dummies.head()

,APPROVED,G,NOT RATED,PASSED,PG,PG-13,R,UNRATED
0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0,0


In [48]:
#make actors CountV with groups of 2 and 3 minimum frequency of 2
countv_1 = CountVectorizer(ngram_range=(2, 3), min_df=2)

actors_cv = countv_1.fit_transform(movie_data["Actors"])

In [51]:
#turn into df with column names
actors_cv_df = pd.DataFrame(actors_cv.todense(), columns=countv_1.get_feature_names())
actors_cv_df.head()

,aamir khan,adrien brody,al pacino,al pacino robert,alec guinness,alexandra maria,alexandra maria lara,anatoliy solonitsyn,anne moss,anthony quinn,arnold schwarzenegger,bale michael,bale michael caine,ben affleck,ben kingsley,benicio del,benicio del toro,bette davis,bibi andersson,billy dee,billy dee williams,brad pitt,bruce willis,carrie anne,carrie anne moss,carrie fisher,carrie fisher billy,chapman john,chapman john cleese,charles chaplin,charles vanel,charlton heston,christian bale,christian bale michael,christoph waltz,christopher plummer,christopher walken,claude rains,claudia cardinale,clint eastwood,clint eastwood lee,daniel day,daniel day lewis,day lewis,de niro,dee williams,del toro,diane keaton,eastwood lee,eastwood lee van,...,martin sheen,matt damon,matthew mcconaughey,max von,max von sydow,michael biehn,michael caine,morgan freeman,murray abraham,natalie portman,newman robert,newman robert redford,nikolay grinko,noel appleby,orson welles,pacino robert,patrick magee,paul newman,paul newman robert,paul reiser,peter lorre,ralph fiennes,richard attenborough,robert de,robert de niro,robert duvall,robert redford,robert shaw,russell crowe,sala baker,sean astin,sharman joshi,sigourney weaver,steve buscemi,takashi shimura,tatsuya nakadai,terry gilliam,thomas mitchell,tim allen,tim roth,tom hanks,tom hanks tim,tom hardy,toni collette,toshirô mifune,van cleef,viola davis,von sydow,willem dafoe,william holden
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
#apply CountVectorizer to genre:
countv_2 = CountVectorizer()
genre_cv = countv_2.fit_transform(movie_data["Genre"])

In [54]:
genre_cv_df = pd.DataFrame(genre_cv.todense(), columns=countv_2.get_feature_names())
genre_cv_df.head()

,action,adventure,animation,biography,comedy,crime,drama,family,fantasy,fi,film,history,horror,music,musical,mystery,noir,romance,sci,sport,thriller,war,western
0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
year_dummies = pd.get_dummies(movie_data["Year"])
year_dummies.head()

,1921,1925,1926,1927,1928,1931,1934,1936,1939,1940,1941,1942,1944,1946,1948,1949,1950,1951,1952,1953,1954,1955,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
X = pd.DataFrame(movie_data["Runtime"]).copy() #add in any other columns that don't need changes
X = X.join(dir_dummies)
X = X.join(rated_dummies)
X = X.join(actors_cv_df)
X = X.join(genre_cv_df)
X = X.join(year_dummies)

In [60]:
X.head()

,Runtime,"Dir_Aamir Khan, Amole Gupte",Dir_Adam Elliot,Dir_Akira Kurosawa,Dir_Alejandro G. Iñárritu,Dir_Alfred Hitchcock,Dir_Andrei Tarkovsky,Dir_Andrew Stanton,"Dir_Andrew Stanton, Lee Unkrich",Dir_Anurag Kashyap,Dir_Asghar Farhadi,Dir_Ashutosh Gowariker,Dir_Billy Wilder,Dir_Brian De Palma,Dir_Bryan Singer,"Dir_Byron Howard, Rich Moore, Jared Bush",Dir_Carl Theodor Dreyer,Dir_Carol Reed,Dir_Chan-wook Park,Dir_Charles Chaplin,Dir_Christopher Nolan,Dir_Clint Eastwood,"Dir_Clyde Bruckman, Buster Keaton",Dir_Curtis Hanson,Dir_Damien Chazelle,Dir_Damián Szifrón,Dir_Danny Boyle,Dir_Darren Aronofsky,Dir_David Fincher,Dir_David Lean,Dir_David Lynch,Dir_David Yates,"Dir_Dean DeBlois, Chris Sanders",Dir_Denis Villeneuve,Dir_Elem Klimov,Dir_Elia Kazan,"Dir_Ethan Coen, Joel Coen",Dir_F.W. Murnau,Dir_Federico Fellini,"Dir_Fernando Meirelles, Kátia Lund",Dir_Florian Henckel von Donnersmarck,Dir_Francis Ford Coppola,Dir_Frank Capra,Dir_Frank Darabont,Dir_François Truffaut,Dir_Fritz Lang,Dir_Gavin O'Connor,Dir_George Lucas,Dir_George Miller,Dir_George Roy Hill,...,1967,1968,1969,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,142,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,175,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,202,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
#5) train/test split and run random forest model
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [63]:
kfolds = StratifiedKFold()
for train, test in kfolds.split(X, is_amazing):
    rf = RandomForestClassifier()
    rf.fit(X.iloc[train], is_amazing.iloc[train])
    print (rf.score(X.iloc[test], is_amazing.iloc[test]))
    actual_values = is_amazing.iloc[test]
    predicted_values = rf.predict(X.iloc[test])
    #add confusion matrices and classification reports
    print(confusion_matrix(actual_values, predicted_values))
    print(classification_report(actual_values, predicted_values))

0.619047619048
[[48  4]
 [28  4]]
             precision    recall  f1-score   support

          0       0.63      0.92      0.75        52
          1       0.50      0.12      0.20        32

avg / total       0.58      0.62      0.54        84

0.595238095238
[[45  7]
 [27  5]]
             precision    recall  f1-score   support

          0       0.62      0.87      0.73        52
          1       0.42      0.16      0.23        32

avg / total       0.55      0.60      0.54        84

0.670731707317
[[50  1]
 [26  5]]
             precision    recall  f1-score   support

          0       0.66      0.98      0.79        51
          1       0.83      0.16      0.27        31

avg / total       0.72      0.67      0.59        82



In [64]:
rf_fd = RandomForestClassifier()
rf_fd.fit(X, is_amazing)
print (rf.score(X, is_amazing))
actual_values_fd = is_amazing
predicted_values_fd = rf.predict(X)
print(confusion_matrix(actual_values_fd, predicted_values_fd))
print(classification_report(actual_values_fd, predicted_values_fd))

0.884
[[154   1]
 [ 28  67]]
             precision    recall  f1-score   support

          0       0.85      0.99      0.91       155
          1       0.99      0.71      0.82        95

avg / total       0.90      0.88      0.88       250



In [74]:
feature_importances = pd.DataFrame(rf_fd.feature_importances_, index = X.columns, columns=['importance']).sort_values("importance", ascending=False)
feature_importances.head(50)

,importance
Runtime,0.072402
1994,0.021076
Dir_Christopher Nolan,0.017542
Dir_Robert Zemeckis,0.015819
drama,0.014008
1997,0.013932
tim roth,0.013519
mystery,0.012986
joseph gordon,0.011406
R,0.011059


In [72]:
#make a bar graph of the top 20 feature importances with ticks for each name

#APOLOGIES--I ran short on time for the visualization part...will try to sneak in before grading
